In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
import itertools
import wrangle as w

# DBSCAN import
from sklearn.cluster import DBSCAN

# Scaler import
from sklearn.preprocessing import MinMaxScaler

Imports Successful


In [2]:
directory='/Users/chellyannmoreno/codeup-data-science/project_anomaly/'
filename='logs.csv'

In [3]:
df=w.get_logs(directory,filename)

In [4]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2


In [6]:
df['access_time'] = df.apply(lambda row: str(row['date']) + str(row['time']), axis=1)

In [7]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,access_time
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,2018-01-2609:55:03
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,2018-01-2609:56:02
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,2018-01-2609:56:05
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,2018-01-2609:56:06
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2,2018-01-2609:56:24


In [8]:
df.shape

(847330, 16)

In [9]:
df=df.rename(columns={'name':'cohort','created_at':'created','updated_at':'updated','date':'access_date'})

In [10]:
df.head()

,access_date,time,path,user_id,cohort_id,ip,id,cohort,slack,start_date,end_date,created,updated,deleted_at,program_id,access_time
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,2018-01-2609:55:03
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,2018-01-2609:56:02
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,2018-01-2609:56:05
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,2018-01-2609:56:06
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2,2018-01-2609:56:24


In [11]:
df=df.drop(columns={'slack','deleted_at'})

In [12]:
df.head()

,access_date,time,path,user_id,cohort_id,ip,id,cohort,start_date,end_date,created,updated,program_id,access_time
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,2018-01-2609:55:03
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,2018-01-2609:56:02
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,2018-01-2609:56:05
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,2018-01-2609:56:06
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,2018-01-2609:56:24


In [12]:
df.start_date=df.start_date.astype('datetime64')
df.end_date=df.end_date.astype('datetime64')
df.created=df.created.astype('datetime64')
df.updated=df.updated.astype('datetime64')
df.access_date=df.access_date.astype('datetime64')
df.time=df.time.astype('datetime64')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73739 entries, 0 to 73738
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           73739 non-null  int64         
 1   cohort       73739 non-null  object        
 2   start_date   73739 non-null  datetime64[ns]
 3   end_date     73739 non-null  datetime64[ns]
 4   created      73739 non-null  datetime64[ns]
 5   updated      73739 non-null  datetime64[ns]
 6   program_id   73739 non-null  int64         
 7   access_date  73739 non-null  datetime64[ns]
 8   time         73739 non-null  datetime64[ns]
 9   path         73739 non-null  object        
 10  user_id      73739 non-null  int64         
 11  cohort_id    72405 non-null  float64       
 12  ip           73739 non-null  object        
 13  access_time  73739 non-null  object        
dtypes: datetime64[ns](6), float64(1), int64(3), object(4)
memory usage: 8.4+ MB


In [14]:
df.isnull().sum()

id                0
cohort            0
start_date        0
end_date          0
created           0
updated           0
program_id        0
access_date       0
time              0
path              0
user_id           0
cohort_id      1334
ip                0
access_time       0
dtype: int64

In [15]:
df.program_id.value_counts()

1    37794
2    31980
3     2458
4     1507
Name: program_id, dtype: int64

In [16]:
df.cohort_id.nunique()

10

In [17]:
df.cohort_id.value_counts()

28.0    25097
22.0    23840
14.0     8401
23.0     5612
21.0     3413
1.0      3252
8.0      1617
13.0      592
18.0      474
16.0      107
Name: cohort_id, dtype: int64

In [18]:
df.program_id.value_counts()

1    37794
2    31980
3     2458
4     1507
Name: program_id, dtype: int64

In [ ]:
df.cohort_id.isnull().value_counts(normalize=True)

In [ ]:
df['program'] = df.program_id.map({1:'data science',2:'web dev',3:'web dev',4:'web dev'})

In [ ]:
df.head()

In [ ]:
df.access_date=df.access_date.astype('datetime64')
    df.time=df.time.astype('datetime64')

In [ ]:
def prep_logs(df):
    df['access_date'] = df.apply(lambda row: str(row['date']) + ' ' + str(row['time']), axis=1)
    df=df.drop(columns={'Unnamed: 0','id','slack','deleted_at','date','time'})
    df=df.rename(columns={'name':'cohort','created_at':'created','updated_at':'updated'})
    df.start_date=df.start_date.astype('datetime64')
    df.end_date=df.end_date.astype('datetime64')
    df.created=df.created.astype('datetime64')
    df.updated=df.updated.astype('datetime64')
    df.access_date=df.access_date.astype('datetime64')
    df['program'] = df.program_id.map({1:'data science',2:'web dev',3:'web dev',4:'web dev'})
    cohort_id_mapping = {'Bash': 2, 'Darden': 3, 'Florence': 4,'Hyperion':5,'Jupiter':6}
    df['cohort_id'].fillna(df['cohort'].map(cohort_id_mapping), inplace=True)
    return df


In [ ]:
df['lesson'] = df['path'].str.split('/').str[-2]

In [ ]:
 df['endpoint'] = df['path'].str.split('/').str[-1]

In [ ]:
df.program_id.value_counts()

In [ ]:
program_1=df[df.program_id==1]
program_1.sample(40)

In [ ]:
program_1.lesson.value_counts()

In [ ]:
program_1.endpointb.value_counts()

In [ ]:
program_1.start_date.value_counts().sort_index()

In [ ]:
program_2=df[df.program_id==2]
program_2.sample(30)

In [ ]:
program_2=df[df.program_id==2]
program_2.lesson.value_counts()

In [ ]:
program_2.endpoint.value_counts()

In [ ]:
program_2.start_date.value_counts().sort_index()

In [ ]:
program_3=df[df.program_id==3]
program_3.sample(20)

In [ ]:
program_3=df[df.program_id==3]
program_3.lesson.value_counts()

In [ ]:
program_3.endpoint.value_counts()

In [ ]:
program_3.start_date.value_counts().sort_index()

In [ ]:
program_4=df[df.program_id==4]
program_4.lesson.value_counts()

In [ ]:
program_4.endpoint.value_counts()

In [ ]:
program_4.start_date.value_counts().sort_index()

In [ ]:
df.cohort.nunique()

In [ ]:
df.cohort_id.nunique()

In [ ]:
df.groupby(['cohort']).sum()

In [ ]:
df.cohort_id.value_counts()

In [ ]:
df=a.get_logs(directory,filename)

In [ ]:
df=prep_logs(df)

In [ ]:
df.head()

In [ ]:
df.cohort_id.value_counts()

In [ ]:
cohort_id_mapping = {'Bash': 2, 'Darden': 3, 'Florence': 4,'Hyperion':5,'Jupiter':6}

In [ ]:
df['cohort_id'].fillna(df['cohort'].map(cohort_id_mapping), inplace=True)

In [ ]:
df.cohort_id.value_counts()

In [ ]:
df.shape

In [ ]:
program_4.access_date.value_counts().sort_index()